In [1]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [3]:
books["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Human-animal relationships,1
475,Imperialism,1
476,Aged women,1
477,Humorous stories,1


In [4]:
category_mapping = {
    'Fiction': 'Fiction',
    'Juvenile Fiction': "Children's Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Science': "Nonfiction",
    'Poetry': "Fiction"
}

books["simple_categories"] = books["categories"].map(category_mapping)

In [5]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN


In [6]:
books[~(books["simple_categories"].isna())].shape

(3743, 14)

In [7]:
from transformers import pipeline

fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification",
                model="facebook/bart-large-mnli",
                device = "cuda" )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


In [8]:
sequence = books.loc[books["simple_categories"]=="Fiction", "description"].reset_index(drop=True)[0]

In [9]:
pipe(sequence,fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [11]:
import numpy as np
max_index = np.argmax(pipe(sequence,fiction_categories)["scores"])
max_label = pipe(sequence,fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [13]:
def generate_predictions(sequence,categories):
  predictions = pipe(sequence,categories)
  max_index = np.argmax(predictions["scores"])
  max_label = predictions["labels"][max_index]
  return max_label

In [15]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0,300)):
  sequence = books.loc[books["simple_categories"]=="Fiction", "description"].reset_index(drop=True)[i]
  predicted_cats += [generate_predictions(sequence,fiction_categories)]
  actual_cats += ["Fiction"]

100%|██████████| 300/300 [00:26<00:00, 11.36it/s]


In [16]:
for i in tqdm(range(0,300)):
  sequence = books.loc[books["simple_categories"]=="Nonfiction", "description"].reset_index(drop=True)[i]
  predicted_cats += [generate_predictions(sequence,fiction_categories)]
  actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [00:28<00:00, 10.48it/s]


In [17]:
predictions_df = pd.DataFrame({"actual_categories":actual_cats,"predicted_categories":predicted_cats})

In [19]:
predictions_df.head()

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction


In [20]:
predictions_df["correct_prediction"] = (np.where(predictions_df["actual_categories"]==predictions_df["predicted_categories"],1,0))

In [22]:
predictions_df["correct_prediction"].sum()/len(predictions_df)

np.float64(0.7783333333333333)

In [23]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books["simple_categories"].isna(),["isbn13","description"]].reset_index(drop=True)

In [24]:
for i in tqdm(range(0,len(missing_cats))):
  sequence = missing_cats['description'][i]
  predicted_cats += [generate_predictions(sequence,fiction_categories)]
  isbns += [missing_cats['isbn13'][i]]

100%|██████████| 1454/1454 [02:16<00:00, 10.63it/s]


In [35]:
missing_predicted_df = pd.DataFrame({"isbn13":isbns,"predicted_categories":predicted_cats})

In [36]:
missing_predicted_df.head()

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction


In [37]:
books = pd.merge(books,missing_predicted_df,on="isbn13",how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(),books["predicted_categories"], books["simple_categories"])
books = books.drop(columns=["predicted_categories"])

In [50]:
books.shape

(5197, 14)

In [44]:
books.isna().sum()

,0
isbn13,0
isbn10,0
title,0
authors,32
categories,30
thumbnail,166
description,0
published_year,0
average_rating,0
num_pages,0


In [46]:
books.nunique()

,0
isbn13,5197
isbn10,5197
title,4969
authors,3045
categories,479
thumbnail,5031
description,5154
published_year,83
average_rating,190
num_pages,808


In [51]:
books.to_csv("books_with_categories.csv",index=False)